In [1]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols

In [2]:
t = dynamicsymbols._t


x, y, theta, phi = dynamicsymbols('x y theta phi') # x and y -> buoy coordinates

m_b, m_d, l, g = sp.symbols('m_b m_d l g', positive = True) # mass buoy, mass drogue in [kg], wire length in [m], g in [m(s^2)]

k_b, k_d = sp.symbols('k_b k_d', positive = True) # drag coefficients for buoy and drogue

U_0, V_0 = sp.symbols('U_0 V_0', real = True) # surface horizontal velocities

In [3]:
r_b = sp.Matrix([x, y, 0]) # buoy position

r = l * sp.Matrix( # same as before?
    [
        sp.sin(theta) * sp.cos(phi),
        sp.sin(theta) * sp.sin(phi),
        sp.cos(theta),
    ]
)

r_d = r_b + r # drogue position

display(r_d)


Matrix([
[l*sin(theta(t))*cos(phi(t)) + x(t)],
[l*sin(phi(t))*sin(theta(t)) + y(t)],
[                   l*cos(theta(t))]])

In [4]:
# velocities 

v_b = r_b.diff(t)
v_d = r_d.diff(t) # why not blue

u_b = sp.Matrix([U_0,V_0,0])

In [5]:
def _mag(vec):
    return sp.sqrt(vec.dot(vec))

F_b = -k_b * _mag(v_b - u_b) * (v_b - u_b) # drag buoy with ext. velocity
F_d = -k_d * _mag(v_d) * v_d # drag drogue

In [6]:
F_b

Matrix([
[-k_b*(-U_0 + Derivative(x(t), t))*sqrt((-U_0 + Derivative(x(t), t))**2 + (-V_0 + Derivative(y(t), t))**2)],
[-k_b*(-V_0 + Derivative(y(t), t))*sqrt((-U_0 + Derivative(x(t), t))**2 + (-V_0 + Derivative(y(t), t))**2)],
[                                                                                                        0]])

In [7]:
sp.simplify(F_d.subs({theta:sp.pi}))



Matrix([
[-k_d*sqrt(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)*Derivative(x(t), t)],
[-k_d*sqrt(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)*Derivative(y(t), t)],
[                                                                             0]])

In [8]:
# lagrangian

T = sp.Rational(1, 2) * m_b * v_b.dot(v_b) + sp.Rational(1, 2) * m_d * v_d.dot(v_d) # kinetic energy

V = m_d * g * r_d[2]

L = T - V

T

m_b*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m_d*(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))**2 + (l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t) + Derivative(y(t), t))**2)/2

In [9]:
q = sp.Matrix([x, y, theta, phi])
qd = q.diff(t)
qdd = qd.diff(t)

In [10]:
Q = sp.Matrix([r_b.diff(qi).dot(F_b) + r_d.diff(qi).dot(F_d) for qi in q])

Q = sp.simplify(Q)

Q

Matrix([
[k_b*(U_0 - Derivative(x(t), t))*sqrt((U_0 - Derivative(x(t), t))**2 + (V_0 - Derivative(y(t), t))**2) - k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))**2 + (l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t) + Derivative(y(t), t))**2)*(-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))],
[ k_b*(V_0 - Derivative(y(t), t))*sqrt((U_0 - Derivative(x(t), t))**2 + (V_0 - Derivative(y(t), t))**2) - k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))**2 + (l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t) + Derivative(y(t), t))**

In [11]:
eoms = sp.Matrix(
    [
        L.diff(qdj).diff(t)
        - L.diff(qj)
        - Qj
        for qj, qdj, Qj in zip(q, qd, Q)
    ]
)
eoms = sp.simplify(eoms)
eoms

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       -k_b*(U_0 - Derivative(x(t), t))*sqrt((U_0 - Derivative(x(t), t))**2 + (V_0 - Derivative(y(t), t))**2) + k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Deriva

In [37]:
xx, yy, phi_1 = dynamicsymbols('xx yy phi_1')
UU, VV = sp.symbols('UU VV', real = True)

In [32]:
xx

xx(t)

In [38]:
sp.simplify(eoms[0].subs({x:xx, y:yy, U_0:UU, V_0:VV, phi:phi_1})-eoms[1].subs({x:yy, y:xx, U_0:VV, V_0:UU, phi:-phi_1}))

k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi_1(t))*sin(theta(t))*Derivative(phi_1(t), t) + l*cos(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(xx(t), t))**2 + (l*sin(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(phi_1(t))*Derivative(phi_1(t), t) + Derivative(yy(t), t))**2)*(-l*sin(phi_1(t))*sin(theta(t))*Derivative(phi_1(t), t) + l*cos(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(xx(t), t)) - k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi_1(t))*sin(theta(t))*Derivative(phi_1(t), t) + l*cos(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(yy(t), t))**2 + (-l*sin(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) - l*sin(theta(t))*cos(phi_1(t))*Derivative(phi_1(t), t) + Derivative(xx(t), t))**2)*(-l*sin(phi_1(t))*cos(theta(t))*Derivative(theta(t), t) - l*sin(theta(t))*cos(phi_1(t))*Derivative(phi_1(t), t) + Derivative(xx(t), t)) - m_d*(l*sin(phi_1(t))*sin(theta(t))*Derivati

In [12]:
M, F = sp.linear_eq_to_matrix(sp.simplify(eoms), list(qdd))

In [13]:
F

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       k_b*(U_0 - Derivative(x(t), t))*sqrt((U_0 - Derivative(x(t), t))**2 + (V_0 - Derivative(y(t), t))**2) - k_d*sqrt(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))**2 + (l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(ph

In [14]:
M

Matrix([
[                                                 m_b + m_d,                                                          0, l*m_d*cos(phi(t))*cos(theta(t)), -l*m_d*sin(phi(t))*sin(theta(t))],
[                                                         0,                                                  m_b + m_d, l*m_d*sin(phi(t))*cos(theta(t)),  l*m_d*sin(theta(t))*cos(phi(t))],
[                           l*m_d*cos(phi(t))*cos(theta(t)),                            l*m_d*sin(phi(t))*cos(theta(t)),                        l**2*m_d,                                0],
[l*m_d*(-cos(phi(t) - theta(t)) + cos(phi(t) + theta(t)))/2, l*m_d*(-sin(phi(t) - theta(t)) + sin(phi(t) + theta(t)))/2,                               0, l*m_d*(-l*cos(2*theta(t)) + l)/2]])

In [15]:
M.shape, F.shape, qdd.shape


((4, 4), (4, 1), (4, 1))

In [16]:
import numpy as np

In [17]:
par_syms = (U_0, V_0, m_b, m_d, l, g, k_b, k_d)

In [18]:
par_syms

(U_0, V_0, m_b, m_d, l, g, k_b, k_d)

In [19]:
M_func = sp.lambdify((t, q, qd) + par_syms, M, modules='numpy')
F_func = sp.lambdify((t, q, qd) + par_syms, F, modules='numpy')

In [20]:
M_func?

Signature: M_func(t, _Dummy_68, _Dummy_69, U_0, V_0, m_b, m_d, l, g, k_b, k_d)
Docstring:
Created with lambdify. Signature:

func(t, q, qd, U_0, V_0, m_b, m_d, l, g, k_b, k_d)

Expression:

Matrix([[m_b + m_d, 0, l*m_d*cos(phi(t))*cos(theta(t)),...

Source code:

def _lambdifygenerated(t, _Dummy_68, _Dummy_69, U_0, V_0, m_b, m_d, l, g, k_b, k_d):
    [_Dummy_65, _Dummy_64, _Dummy_66, _Dummy_67] = _Dummy_68
    [_Dummy_61, _Dummy_60, _Dummy_62, _Dummy_63] = _Dummy_69
    return array([[m_b + m_d, 0, l*m_d*cos(_Dummy_66)*cos(_Dummy_67), -l*m_d*sin(_Dummy_66)*sin(_Dummy_67)], [0, m_b + m_d, l*m_d*sin(_Dummy_67)*cos(_Dummy_66), l*m_d*sin(_Dummy_66)*cos(_Dummy_67)], [l*m_d*cos(_Dummy_66)*cos(_Dummy_67), l*m_d*sin(_Dummy_67)*cos(_Dummy_66), l**2*m_d, 0], [(1/2)*l*m_d*(-cos(_Dummy_66 - _Dummy_67) + cos(_Dummy_66 + _Dummy_67)), (1/2)*l*m_d*(sin(_Dummy_66 - _Dummy_67) + sin(_Dummy_66 + _Dummy_67)), 0, (1/2)*l*m_d*(-l*cos(2*_Dummy_66) + l)]])


Imported modules:
File:      c:\users\merle\desktop

In [21]:
q


Matrix([
[    x(t)],
[    y(t)],
[theta(t)],
[  phi(t)]])

In [22]:
M_num = M_func(0, [0, 0, np.pi, 0], [0, 0, 0, 0], U_0 = 0, V_0 = 1, m_b = 0.5, m_d = 0.5, l = 3, g = 9.81, k_b = 1, k_d = 2)

In [23]:
M_num

array([[ 1.0000000e+00,  0.0000000e+00, -1.5000000e+00, -0.0000000e+00],
       [ 0.0000000e+00,  1.0000000e+00, -0.0000000e+00,  1.8369702e-16],
       [-1.5000000e+00, -0.0000000e+00,  4.5000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  1.8369702e-16,  0.0000000e+00,  0.0000000e+00]])

In [24]:
F_num = F_func(0, [0, 0, np.pi, 0], [0, 0, 0, 0], U_0 = 0, V_0 = 1, m_b = 0.5, m_d = 0.5, l = 3, g = 9.81, k_b = 1, k_d = 2)


In [25]:
F_num

array([[ 0.00000000e+00],
       [ 1.00000000e+00],
       [ 1.80206776e-15],
       [-0.00000000e+00]])

In [26]:
np.linalg.solve(M_num, F_num)

array([[ 1.20137851e-15],
       [-9.47621422e-17],
       [ 8.00919007e-16],
       [ 5.44374645e+15]])

In [27]:
np.linalg.solve(M_num, F_num)

array([[ 1.20137851e-15],
       [-9.47621422e-17],
       [ 8.00919007e-16],
       [ 5.44374645e+15]])

In [28]:
q

Matrix([
[    x(t)],
[    y(t)],
[theta(t)],
[  phi(t)]])